In [ ]:
# Import from meg_qc, relative to the path of this file

# add meg_qc module to the path

from meg_qc.meg_qc_pipeline import make_derivative_meg_qc

config_file_path = 'meg_qc/settings.ini'
raw, raw_cropped_filtered_resampled, QC_derivs, QC_simple, df_head_pos, head_pos = make_derivative_meg_qc(config_file_path)

In [ ]:
import mne
picks_EOG = mne.pick_types(raw.info, eog=True)
eog_ch_name = [raw.info['chs'][name]['ch_name'] for name in picks_EOG]
eog_ch_name

In [ ]:
df_head_pos


In [ ]:
bool('False')

#convert "false" to boolean:
import ast
t = ast.literal_eval('False')
t
